In [1]:
import eland as ed
import pandas as pd
import matplotlib.pyplot as plt

# Connect to an Elasticsearch instance
from elasticsearch import Elasticsearch
es = Elasticsearch(
  ['http://localhost:9200'],
  http_auth=("es_kbn", "changeme")
)

In [2]:
tx_6m = ed.DataFrame(es, es_index_pattern="es-invoices-6m").to_pandas()

tx_user = pd.DataFrame(tx_6m['customer_id'].unique())
tx_user.columns = ['customer_id']

#create a dataframe with customer id and first purchase date in tx_next
tx_next_first_purchase = ed.DataFrame(es, es_index_pattern="es-customers-3m").to_pandas()

#create a dataframe with customer id and last purchase date in tx_6m
tx_last_purchase = ed.DataFrame(es, es_index_pattern="es-customers-6m").to_pandas()

#merge two dataframes
tx_purchase_dates = pd.merge(tx_last_purchase,tx_next_first_purchase,on='customer_id',how='left')

#calculate the time difference in days:
tx_purchase_dates['next_purchase_day'] = (tx_purchase_dates['min_purchase_date'] - tx_purchase_dates['max_purchase_date']).dt.days

#merge with tx_user
tx_purchase_dates['customer_id'] = tx_purchase_dates['customer_id'].astype(int)
tx_user = pd.merge(tx_user, tx_purchase_dates[['customer_id','next_purchase_day']],on='customer_id',how='left')

#print tx_user
tx_user.head()

,customer_id,next_purchase_day
0,17664,NaN
1,17663,43.0
2,17659,14.0
3,17658,NaN
4,17624,NaN


In [3]:
#fill NA values with 999
tx_user = tx_user.fillna(999)

#print tx_user
tx_user.head()

,customer_id,next_purchase_day
0,17664,999.0
1,17663,43.0
2,17659,14.0
3,17658,999.0
4,17624,999.0


In [4]:
# Load the data into elasticsearch
ed_users = ed.pandas_to_eland(
    pd_df=tx_user,
    es_client=es,

    # Where the data will live in Elasticsearch
    es_dest_index="es-users",

    # Type overrides for certain columns, this can be used to customize index mapping before ingest
    es_type_overrides={
        "invoice_id": "keyword",
        "next_purchase_day": "integer"
    },

    # If the index already exists what should we do?
    es_if_exists="replace",

    # Wait for data to be indexed before returning
    es_refresh=True,
)
ed_users.info()

<class 'eland.dataframe.DataFrame'>
Index: 2992 entries, 0 to 2991
Data columns (total 2 columns):
 #   Column             Non-Null Count  Dtype
---  ------             --------------  -----
 0   customer_id        2992 non-null   int64
 1   next_purchase_day  2992 non-null   int64
dtypes: int64(2)
memory usage: 64.0 bytes


In [5]:
#RFM Calculations
tx_user = ed.DataFrame(es, es_index_pattern="es-users").to_pandas()
customers = ed.DataFrame(es, es_index_pattern="es-customers-6m").to_pandas()
customers['customer_id'] = customers['customer_id'].astype(int)
tx_user = pd.merge(tx_user, customers[['customer_id','recency','count_orders','total_revenue']], on='customer_id')
tx_user

,customer_id,next_purchase_day,recency,count_orders,total_revenue
0,17664.0,999.0,20,66,7506.200006
1,17663.0,43.0,6,129,18651.299901
2,17659.0,14.0,7,212,23740.800010
3,17658.0,999.0,19,32,4580.899989
4,17624.0,999.0,25,75,9529.399958
...,...,...,...,...,...
2987,17498.0,999.0,12,17,1032.400017
2988,17495.0,999.0,12,27,3221.899990
2989,17509.0,999.0,14,369,54487.000170
2990,17506.0,999.0,14,16,1138.000010


In [6]:
from sklearn.cluster import KMeans
#clustering for Recency
kmeans = KMeans(n_clusters=4)
kmeans.fit(tx_user[['recency']])
tx_user['recency_cluster'] = kmeans.predict(tx_user[['recency']])

#order cluster method
def order_cluster(cluster_field_name, target_field_name,df,ascending):
    new_cluster_field_name = 'new_' + cluster_field_name
    df_new = df.groupby(cluster_field_name)[target_field_name].mean().reset_index()
    df_new = df_new.sort_values(by=target_field_name,ascending=ascending).reset_index(drop=True)
    df_new['index'] = df_new.index
    df_final = pd.merge(df,df_new[[cluster_field_name,'index']], on=cluster_field_name)
    df_final = df_final.drop([cluster_field_name],axis=1)
    df_final = df_final.rename(columns={"index":cluster_field_name})
    return df_final


#order recency clusters
tx_user = order_cluster('recency_cluster', 'recency',tx_user,False)

#print cluster characteristics
tx_user.groupby('recency_cluster')['recency'].describe()


,count,mean,std,min,25%,50%,75%,max
recency_cluster,,,,,,,,
0,466.0,161.603004,15.752628,133.0,144.00,166.5,174.75,181.0
1,538.0,103.249071,15.376626,80.0,89.25,102.0,118.00,132.0
2,756.0,56.633598,11.591131,39.0,46.00,55.5,67.00,79.0
3,1232.0,15.014610,9.198936,0.0,7.00,14.0,22.00,35.0


In [7]:
#k-means
kmeans = KMeans(n_clusters=4)
kmeans.fit(tx_user[['count_orders']])
tx_user['frequency_cluster'] = kmeans.predict(tx_user[['count_orders']])

#order the frequency cluster
tx_user = order_cluster('frequency_cluster', 'count_orders',tx_user,True)

#see details of each cluster
tx_user.groupby('frequency_cluster')['count_orders'].describe()

,count,mean,std,min,25%,50%,75%,max
frequency_cluster,,,,,,,,
0,2643.0,36.425653,31.185789,1.0,12.5,26.0,53.50,136.0
1,320.0,236.390625,98.626527,137.0,168.0,203.5,270.25,595.0
2,28.0,964.535714,357.593239,608.0,707.0,858.5,1077.25,2021.0
3,1.0,10104.000000,NaN,10104.0,10104.0,10104.0,10104.00,10104.0


In [8]:
total_revenue = tx_user.filter(['customer_id','total_revenue'])

#k-means
kmeans = KMeans(n_clusters=4)
kmeans.fit(total_revenue)
tx_user['revenue_cluster'] = kmeans.predict(total_revenue)

#order the revenue cluster
tx_user = order_cluster('revenue_cluster', 'total_revenue',tx_user,True)

#see details of each cluster
tx_user.groupby('revenue_cluster')['total_revenue'].describe()

,count,mean,std,min,25%,50%,75%,max
revenue_cluster,,,,,,,,
0,2808.0,6731.384155,11901.392070,-363218.004614,2172.000000,4401.000002,9331.575039,3.410100e+04
1,164.0,61543.615249,28601.151203,34622.399963,41695.700100,50736.549900,72519.774924,1.704938e+05
2,16.0,297943.150509,77374.875298,198983.099792,259772.975627,280798.749974,334628.603682,5.029020e+05
3,4.0,827010.601580,218624.093847,621149.499636,672733.525359,790105.851940,944382.928162,1.106681e+06


In [9]:
#calculate overall score and use mean() to see details
tx_user['overall_score'] = tx_user['recency_cluster'] + tx_user['frequency_cluster'] + tx_user['revenue_cluster']
score = tx_user.groupby('overall_score').mean().loc[:, ['recency','count_orders','total_revenue']]
score

,recency,count_orders,total_revenue
overall_score,,,
0,161.721239,23.975664,2.581670e+03
1,104.424184,29.589251,4.340444e+03
2,58.700855,40.769231,6.398289e+03
3,18.765226,55.188605,9.015273e+03
4,16.205405,205.362162,3.307154e+04
5,12.464286,297.214286,6.430368e+04
6,11.000000,739.947368,1.590108e+05
7,10.555556,1117.222222,3.816294e+05
8,5.000000,798.000000,1.106681e+06


In [10]:
tx_6m = ed.DataFrame(es, es_index_pattern="es-invoices-6m").to_pandas()
#create a dataframe with CustomerID and Invoice Date
tx_day_order = tx_6m.loc[:, ['customer_id','invoice_date']]
#convert Invoice Datetime to day
tx_day_order['invoice_day'] = tx_6m['invoice_date'].dt.date
tx_day_order = tx_day_order.sort_values(['customer_id','invoice_date'])
#drop duplicates
tx_day_order = tx_day_order.drop_duplicates(subset=['customer_id','invoice_day'],keep='first')
tx_day_order

,customer_id,invoice_date,invoice_day
406827,12346,2019-01-18 10:01:00,2019-01-18
406645,12347,2018-12-07 14:57:00,2018-12-07
406676,12347,2019-01-26 14:30:00,2019-01-26
406705,12347,2019-04-07 10:43:00,2019-04-07
406614,12348,2018-12-16 19:09:00,2018-12-16
...,...,...,...
172,18283,2019-02-28 10:30:00,2019-02-28
202,18283,2019-04-21 16:37:00,2019-04-21
258,18283,2019-05-23 11:33:00,2019-05-23
406829,18287,2018-12-01 11:52:00,2018-12-01


In [11]:
#shifting last 3 purchase dates
tx_day_order['prev_invoice_date'] = tx_day_order.groupby('customer_id')['invoice_day'].shift(1)
tx_day_order['t2_invoice_date'] = tx_day_order.groupby('customer_id')['invoice_day'].shift(2)
tx_day_order['t3_invoice_date'] = tx_day_order.groupby('customer_id')['invoice_day'].shift(3)
tx_day_order

,customer_id,invoice_date,invoice_day,prev_invoice_date,t2_invoice_date,t3_invoice_date
406827,12346,2019-01-18 10:01:00,2019-01-18,NaN,NaN,NaN
406645,12347,2018-12-07 14:57:00,2018-12-07,NaN,NaN,NaN
406676,12347,2019-01-26 14:30:00,2019-01-26,2018-12-07,NaN,NaN
406705,12347,2019-04-07 10:43:00,2019-04-07,2019-01-26,2018-12-07,NaN
406614,12348,2018-12-16 19:09:00,2018-12-16,NaN,NaN,NaN
...,...,...,...,...,...,...
172,18283,2019-02-28 10:30:00,2019-02-28,2019-01-23,2019-01-06,2018-12-01
202,18283,2019-04-21 16:37:00,2019-04-21,2019-02-28,2019-01-23,2019-01-06
258,18283,2019-05-23 11:33:00,2019-05-23,2019-04-21,2019-02-28,2019-01-23
406829,18287,2018-12-01 11:52:00,2018-12-01,NaN,NaN,NaN


In [12]:
tx_day_order['day_diff'] = (tx_day_order['invoice_day'] - tx_day_order['prev_invoice_date']).dt.days
tx_day_order['day_diff_2'] = (tx_day_order['invoice_day'] - tx_day_order['t2_invoice_date']).dt.days
tx_day_order['day_diff_3'] = (tx_day_order['invoice_day'] - tx_day_order['t3_invoice_date']).dt.days
tx_day_order

,customer_id,invoice_date,invoice_day,prev_invoice_date,t2_invoice_date,t3_invoice_date,day_diff,day_diff_2,day_diff_3
406827,12346,2019-01-18 10:01:00,2019-01-18,NaN,NaN,NaN,NaN,NaN,NaN
406645,12347,2018-12-07 14:57:00,2018-12-07,NaN,NaN,NaN,NaN,NaN,NaN
406676,12347,2019-01-26 14:30:00,2019-01-26,2018-12-07,NaN,NaN,50.0,NaN,NaN
406705,12347,2019-04-07 10:43:00,2019-04-07,2019-01-26,2018-12-07,NaN,71.0,121.0,NaN
406614,12348,2018-12-16 19:09:00,2018-12-16,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
172,18283,2019-02-28 10:30:00,2019-02-28,2019-01-23,2019-01-06,2018-12-01,36.0,53.0,89.0
202,18283,2019-04-21 16:37:00,2019-04-21,2019-02-28,2019-01-23,2019-01-06,52.0,88.0,105.0
258,18283,2019-05-23 11:33:00,2019-05-23,2019-04-21,2019-02-28,2019-01-23,32.0,84.0,120.0
406829,18287,2018-12-01 11:52:00,2018-12-01,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
#can also be done with transformation, policy/enrich, ingest as described earlier
tx_day_diff = tx_day_order.groupby('customer_id').agg({'day_diff': ['mean','std']}).reset_index()
tx_day_diff.columns = ['customer_id', 'day_diff_mean','day_diff_std']
tx_day_diff

,customer_id,day_diff_mean,day_diff_std
0,12346,NaN,NaN
1,12347,60.500000,14.849242
2,12348,55.000000,21.213203
3,12350,NaN,NaN
4,12352,11.333333,5.686241
...,...,...,...
2987,18280,96.000000,NaN
2988,18281,NaN,NaN
2989,18282,NaN,NaN
2990,18283,34.600000,12.481987


In [14]:
#for logstash version : https://www.elastic.co/blog/how-to-find-and-remove-duplicate-documents-in-elasticsearch
tx_day_order_last = tx_day_order.drop_duplicates(subset=['customer_id'],keep='last')
tx_day_order_last

,customer_id,invoice_date,invoice_day,prev_invoice_date,t2_invoice_date,t3_invoice_date,day_diff,day_diff_2,day_diff_3
406827,12346,2019-01-18 10:01:00,2019-01-18,NaN,NaN,NaN,NaN,NaN,NaN
406705,12347,2019-04-07 10:43:00,2019-04-07,2019-01-26,2018-12-07,NaN,71.0,121.0,NaN
406637,12348,2019-04-05 10:47:00,2019-04-05,2019-01-25,2018-12-16,NaN,70.0,110.0,NaN
406524,12350,2019-02-02 16:01:00,2019-02-02,NaN,NaN,NaN,NaN,NaN,NaN
406462,12352,2019-03-22 16:07:00,2019-03-22,2019-03-17,2019-03-01,2019-02-16,5.0,21.0,34.0
...,...,...,...,...,...,...,...,...,...
846,18280,2019-03-07 09:52:00,2019-03-07,2018-12-01,NaN,NaN,96.0,NaN,NaN
407668,18281,2018-12-01 17:06:00,2018-12-01,NaN,NaN,NaN,NaN,NaN,NaN
407655,18282,2018-12-01 17:06:00,2018-12-01,NaN,NaN,NaN,NaN,NaN,NaN
258,18283,2019-05-23 11:33:00,2019-05-23,2019-04-21,2019-02-28,2019-01-23,32.0,84.0,120.0


In [15]:
tx_day_order_last = tx_day_order_last.dropna()
tx_day_order_last = pd.merge(tx_day_order_last, tx_day_diff, on='customer_id')
tx_user = pd.merge(tx_user, tx_day_order_last[['customer_id','day_diff','day_diff_2','day_diff_3','day_diff_mean','day_diff_std']], on='customer_id')

In [16]:
tx_user.columns

Index(['customer_id', 'next_purchase_day', 'recency', 'count_orders',
       'total_revenue', 'recency_cluster', 'frequency_cluster',
       'revenue_cluster', 'overall_score', 'day_diff', 'day_diff_2',
       'day_diff_3', 'day_diff_mean', 'day_diff_std'],
      dtype='object')

In [17]:
# Load the data into elasticsearch
ed_users = ed.pandas_to_eland(
    pd_df=tx_user,
    es_client=es,

    # Where the data will live in Elasticsearch
    es_dest_index="es-users",

    # Type overrides for certain columns, this can be used to customize index mapping before ingest
    es_type_overrides={
        "invoice_id": "keyword",
        "next_purchase_day": "integer"
    },

    # If the index already exists what should we do?
    es_if_exists="replace",

    # Wait for data to be indexed before returning
    es_refresh=True,
)
ed_users.info()

<class 'eland.dataframe.DataFrame'>
Index: 671 entries, 0 to 670
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   count_orders       671 non-null    int64  
 1   customer_id        671 non-null    float64
 2   day_diff           671 non-null    float64
 3   day_diff_2         671 non-null    float64
 4   day_diff_3         671 non-null    float64
 5   day_diff_mean      671 non-null    float64
 6   day_diff_std       671 non-null    float64
 7   frequency_cluster  671 non-null    int64  
 8   next_purchase_day  671 non-null    int64  
 9   overall_score      671 non-null    int64  
 10  recency            671 non-null    int64  
 11  recency_cluster    671 non-null    int64  
 12  revenue_cluster    671 non-null    int64  
 13  total_revenue      671 non-null    float64
dtypes: float64(7), int64(7)
memory usage: 64.0 bytes
